(EBs with 90-100% confidence value including 90 from Yifan's RF algorithm)
<br>
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [1]:
#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#sector you want to classify
sector = 1

read in the prediction as well as the confidence values for each tic in sector 1, based off the RF model

In [3]:
sec1 = pd.read_csv('Sector1/sect1predandprob.csv', index_col=0)

In [4]:
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint

2023/02/28 05:47:09.896228 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2023/02/28 05:47:09.909818 Opening GCS connection...
2023/02/28 05:47:10.098165 Mounting file system "tess-goddard-lcs"...
2023/02/28 05:47:10.186093 File system has been successfully mounted.


In [13]:
data_dir = "/home/jupyter/mountpoint/"
ref = pd.read_csv(data_dir + 'sector1lookup.csv')
eb = sec1[sec1['probone'] >= 0.5]
eb = eb[eb['probone'] < 0.6]

In [14]:
eb

,probzero,probone,pred
350097478,0.402,0.598,1
349478450,0.402,0.598,1
317151025,0.402,0.598,1
53939879,0.402,0.598,1
44565828,0.402,0.598,1
...,...,...,...
290647310,0.500,0.500,0
281630373,0.500,0.500,0
300655691,0.500,0.500,0
126944280,0.500,0.500,0


In [16]:
refebs = ref.loc[ref['TIC_ID'].isin(eb.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [18]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector1/50-60%ebs.pdf") as pdf:
    plt.cla()
    for i in range(len(eb)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

In [6]:
tics = eb.index.values
len(tics)

1156

In [7]:
stars = ref.TIC_ID.isin(tics)
closestarslist = []
for i in range(len(tics)):
    rastar = ref[stars].RA.values[i]
    decstar = ref[stars].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist.append(closestars)
closestarstics = []
for i in range (len(closestarslist)):
    tics = []
    tics.append(ref[stars].TIC_ID.values[i])
    for j in range(len(closestarslist[i][0])):
        tics.append(ref[stars].TIC_ID.values[closestarslist[i][0][j]])
    closestarstics.append(tics)

In [8]:
refebs = ref.loc[ref['TIC_ID'].isin(eb.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [9]:
len(lcs)

1156

In [10]:
tic = []
avgflux = []
    
for i in range(len(lcs)):
    tic.append(refebs['TIC_ID'].values[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [11]:
df = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(df)

In [12]:
highestfluxtics = []
for i in range(len(closestarstics)):
    a = []
    for j in closestarstics[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics[i][a.index(max(a))])

In [13]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [14]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [15]:
newlist = [] #eb list with no duplicates, cut down to ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [17]:
len(newlist)

755